# Building the dataset of research papers

The [Entrez](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html) module, a part of the [Biopython](http://biopython.org/) library, will be used to interface with [PubMed](http://www.ncbi.nlm.nih.gov/pubmed).<br>
You can download Biopython from [here](http://biopython.org/wiki/Download).

In this notebook we will be covering several of the steps taken in the [Biopython Tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html), specifically in [Chapter 9  Accessing NCBI’s Entrez databases](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc109).

In [1]:
%pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [2]:
from Bio import Entrez

# NCBI requires you to set your email address to make use of NCBI's E-utilities
Entrez.email = "r.m.siebes@vu.nl"

The datasets will be saved as serialized Python objects, compressed with bzip2.
Saving/loading them will therefore require the [pickle](http://docs.python.org/3/library/pickle.html) and [bz2](http://docs.python.org/3/library/bz2.html) modules.

In [3]:
import pickle, bz2, os, zipfile

## EInfo: Obtaining information about the Entrez databases

In [4]:
# accessing extended information about the PubMed database
pubmed = Entrez.read( Entrez.einfo(db="pubmed"), validate=False )[u'DbInfo']

# list of possible search fields for use with ESearch:
search_fields = { f['Name']:f['Description'] for f in pubmed["FieldList"] }

In search_fields, we find 'TIAB' ('Free text associated with Abstract/Title') as a possible search field to use in searches.

In [5]:
search_fields

{'ALL': 'All terms from all searchable fields',
 'UID': 'Unique number assigned to publication',
 'FILT': 'Limits the records',
 'TITL': 'Words in title of publication',
 'MESH': 'Medical Subject Headings assigned to publication',
 'MAJR': 'MeSH terms of major importance to publication',
 'JOUR': 'Journal abbreviation of publication',
 'AFFL': "Author's institutional affiliation and address",
 'ECNO': 'EC number for enzyme or CAS registry number',
 'SUBS': 'CAS chemical name or MEDLINE Substance Name',
 'PDAT': 'Date of publication',
 'EDAT': 'Date publication first accessible through Entrez',
 'VOL': 'Volume number of publication',
 'PAGE': 'Page number(s) of publication',
 'PTYP': 'Type of publication (e.g., review)',
 'LANG': 'Language of publication',
 'ISS': 'Issue number of publication',
 'SUBH': 'Additional specificity for MeSH term',
 'SI': 'Cross-reference from publication to other databases',
 'MHDA': 'Date publication was indexed with MeSH terms',
 'TIAB': 'Free text associa

## ESearch: Searching the Entrez databases

To have a look at the kind of data we get when searching the database, we'll perform a search for papers authored by Haasdijk:

In [6]:
example_authors = ['Haasdijk E']
example_search = Entrez.read( Entrez.esearch( db="pubmed", term=' AND '.join([a+'[AUTH]' for a in example_authors]) ) )
example_search

{'Count': '44', 'RetMax': '20', 'RetStart': '0', 'IdList': ['40943785', '40736755', '40701362', '39325889', '38631313', '33501027', '33501026', '33500899', '29311830', '28513205', '28513201', '28323435', '28140628', '26933487', '24977986', '24901702', '24852945', '24708899', '24252306', '23580075'], 'TranslationSet': [{'From': 'Haasdijk E[AUTH]', 'To': 'haasdijk e[Author]'}], 'QueryTranslation': 'haasdijk e[Author]'}

Note how the result being produced is not in Python's native string format:

In [7]:
type( example_search['IdList'][0] )

Bio.Entrez.Parser.StringElement

The part of the query's result we are most interested in is accessible through

In [8]:
example_ids = [ int(id) for id in example_search['IdList'] ]
print(example_ids)

[40943785, 40736755, 40701362, 39325889, 38631313, 33501027, 33501026, 33500899, 29311830, 28513205, 28513201, 28323435, 28140628, 26933487, 24977986, 24901702, 24852945, 24708899, 24252306, 23580075]


### PubMed IDs dataset

We will now assemble a dataset comprised of research articles containing the given keyword, in either their titles or abstracts.

In [9]:
#  Due to the new limit (only 10K max results) the Entrez library is not suitable to fetch all ids for a query
#  - Therefore this has to be done via the more complicated way using the Entrez Direct e-utility:
#     https://www.ncbi.nlm.nih.gov/sites/books/NBK179288/ 
#  - also an API key has to be registered at NCBI
#  - the command used for fetching the ids: 
#               esearch -db pubmed -query "recognition" | efetch -format uid >results.txt
#  - thus, instead of using the output from the script below (recognition_ids.pkl.bz2) we use
search_term = 'pandemic'

In [10]:
Ids_zip = 'data/' + search_term + '_Ids.zip'
Ids_file = search_term + '_Ids.txt'

if os.path.exists( Ids_zip ):
    with zipfile.ZipFile(Ids_zip, 'r') as myzip:
        
        with myzip.open(Ids_file) as myfile:
           Ids_str = myfile.read().decode('UTF-8').splitlines(keepends=True)

        # convert Ids to integers (and ensure that the conversion is reversible)
        Ids = [ int(id) for id in Ids_str ]
else:
     raise Exception('File not found' )



In [11]:
ids_to_remove = [
    40982904
]
for id_to_remove in ids_to_remove:
    try:
        Ids.remove(id_to_remove)
    except ValueError:
        print(f"ID {id_to_remove} not found in the list.")

    
    # Save list of Ids
    pickle.dump( Ids, bz2.BZ2File( Ids_file, 'wb' ) )
    
total = len( Ids )
print('%d documents contain the search term "%s".' % ( total, search_term ) )

76241 documents contain the search term "pandemic".


Taking a look at what we just retrieved, here are the last 5 elements of the `Ids` list:

In [12]:



Ids[:5]

[59080, 128105, 184568, 280169, 293742]

## ESummary: Retrieving summaries from primary IDs

To have a look at the kind of metadata we get from a call to `Entrez.esummary()`, we now fetch the summary of one of Haasdijk's papers (using one of the PubMed IDs we obtained in the previous section:

In [13]:
example_paper = Entrez.read( Entrez.esummary(db="pubmed", id='293742') )[0]

def print_dict( p ):
    for k,v in p.items():
        print(k)
        print('\t', v)

print_dict(example_paper)

Item
	 []
Id
	 293742
PubDate
	 1979 Dec
EPubDate
	 
Source
	 Proc Natl Acad Sci U S A
AuthorList
	 ['Young JF', 'Palese P']
LastAuthor
	 Palese P
Title
	 Evolution of human influenza A viruses in nature: recombination contributes to genetic variation of H1N1 strains.
Volume
	 76
Issue
	 12
Pages
	 6547-51
LangList
	 ['English']
NlmUniqueID
	 7505876
ISSN
	 0027-8424
ESSN
	 1091-6490
PubTypeList
	 ['Journal Article']
RecordStatus
	 PubMed - indexed for MEDLINE
PubStatus
	 ppublish
ArticleIds
	 {'pubmed': ['293742'], 'medline': [], 'pmc': 'PMC411903', 'pmcid': 'pmc-id: PMC411903;', 'doi': '10.1073/pnas.76.12.6547'}
DOI
	 10.1073/pnas.76.12.6547
History
	 {'pubmed': ['1979/12/01 00:00'], 'medline': ['1979/12/01 00:01'], 'entrez': '1979/12/01 00:00', 'pmc-release': '1980/06/01 00:00'}
References
	 []
HasAbstract
	 IntegerElement(1, attributes={})
PmcRefCount
	 IntegerElement(20, attributes={})
FullJournalName
	 Proceedings of the National Academy of Sciences of the United States of Americ

For now, we'll keep just some basic information for each paper: title, list of authors, publication year, and [DOI](https://en.wikipedia.org/wiki/Digital_object_identifier).

In case you are not familiar with the DOI system, know that the paper above can be accessed through the link  `https://www.doi.org/` followed by the paper's DOI.

In [14]:
( example_paper['Title'], example_paper['AuthorList'], int(example_paper['PubDate'][:4]), example_paper['DOI'] )

('Evolution of human influenza A viruses in nature: recombination contributes to genetic variation of H1N1 strains.',
 ['Young JF', 'Palese P'],
 1979,
 '10.1073/pnas.76.12.6547')

### Summaries dataset

We are now ready to assemble a dataset containing the summaries of all the paper `Ids` we previously fetched.

To reduce the memory footprint, and to ensure the saved datasets won't depend on Biopython being installed to be properly loaded, values returned by `Entrez.read()` will be converted to their corresponding native Python types. We start by defining a function for helping with the conversion of strings:

In [15]:
Summaries_file = 'data/' + search_term + '_Summaries.pkl.bz2'


In [16]:
if os.path.exists( Summaries_file ):
    Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
    
else:
    # `Summaries` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Summaries = []
    retrieve_per_query = 500
    
    print('Fetching Summaries of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
            SummariesDict = dict( Summaries )
            print('length=', len(SummariesDict))
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
       # print(query_ids)
        
        s = Entrez.read( Entrez.esummary( db="pubmed", id=query_ids ) )
        
        # out of the retrieved data, we will keep only a tuple (title, authors, year, DOI), associated with the paper's id.
        # (all values converted to native Python formats)
        for p in s:
            
            try:
                f = [
                    ( int( p['Id'] ), (
                        str( p['Title'] ),
                        [ str(a) for a in p['AuthorList'] ],
                        int( p['PubDate'][:4] ),                # keeps just the publication year
                        str( p.get('DOI', '') )            # papers for which no DOI is available get an empty string in their place
                        ) )
                    ]
                #print(int( p['Id'] ))
                Summaries.extend( f )
            except ValueError as e:
                print("\nError with ID " + p['Id'] + ": " + str(e))
                print("Manually remove this ID above and re-run code.")
    
    # Save Summaries, as a dictionary indexed by Ids
    Summaries = dict( Summaries )
    
    pickle.dump( Summaries, bz2.BZ2File( Summaries_file, 'wb' ) )


Let us take a look at the first 3 retrieved summaries:

In [17]:
print(len(Ids))

#print(Summaries[Ids[200001]])
print(len(Summaries))
{ id : Summaries[id] for id in Ids[1:3] }

76241
76241


{128105: ('[Supplementary data on the theory concerning the cyclic evolution of influenza viruses].',
  ['Niculescu IT',
   'Zilişteanu E',
   'Creţescu L',
   'Maţepiuc M',
   'Crainic R',
   'Oprescu E'],
  1975,
  ''),
 184568: ('Acute haemorrhagic conjunctivitis and enterovirus 70 in Kenya.',
  ['Metselaar D', 'Awan AM', 'Ensering HL'],
  1976,
  '')}

## EFetch: Downloading full records from Entrez

`Entrez.efetch()` is the function that will allow us to obtain paper abstracts. Let us start by taking a look at the kind of data it returns when we query PubMed's database.

In [18]:
q = Entrez.read( Entrez.efetch(db="pubmed", id='349', retmode="xml") )['PubmedArticle']

`q` is a list, with each member corresponding to a queried id. Because here we only queried for one id, its results are then in `q[0]`.

In [19]:
type(q), len(q)

(list, 1)

1
At `q[0]` we find a dictionary containing two keys, the contents of which we print below.

In [20]:
type(q[0]), q[0].keys()

(Bio.Entrez.Parser.DictionaryElement,
 dict_keys(['MedlineCitation', 'PubmedData']))

In [21]:
print_dict( q[0][ 'PubmedData' ] )

ReferenceList
	 []
History
	 [DictElement({'Year': '1975', 'Month': '11', 'Day': '1'}, attributes={'PubStatus': 'pubmed'}), DictElement({'Year': '1975', 'Month': '11', 'Day': '1', 'Hour': '0', 'Minute': '1'}, attributes={'PubStatus': 'medline'}), DictElement({'Year': '1975', 'Month': '11', 'Day': '1', 'Hour': '0', 'Minute': '0'}, attributes={'PubStatus': 'entrez'})]
PublicationStatus
	 ppublish
ArticleIdList
	 [StringElement('349', attributes={'IdType': 'pubmed'})]


The key `'MedlineCitation'` maps into another dictionary. In that dictionary, most of the information is contained under the key `'Article'`. To minimize the clutter, below we show the contents of `'MedlineCitation'` excluding its `'Article'` member, and below that we then show the contents of `'Article'`.

In [22]:
print_dict( { k:v for k,v in q[0][ 'MedlineCitation' ].items() if k!='Article' } )

KeywordList
	 []
CitationSubset
	 ['IM']
InvestigatorList
	 []
OtherID
	 []
GeneralNote
	 []
SpaceFlightMission
	 []
OtherAbstract
	 []
PMID
	 349
DateCompleted
	 {'Year': '1976', 'Month': '02', 'Day': '13'}
DateRevised
	 {'Year': '2010', 'Month': '11', 'Day': '18'}
MedlineJournalInfo
	 {'Country': 'United States', 'MedlineTA': 'Heart Lung', 'NlmUniqueID': '0330057', 'ISSNLinking': '0147-9563'}
MeshHeadingList
	 [{'QualifierName': [StringElement('etiology', attributes={'UI': 'Q000209', 'MajorTopicYN': 'N'}), StringElement('metabolism', attributes={'UI': 'Q000378', 'MajorTopicYN': 'N'}), StringElement('therapy', attributes={'UI': 'Q000628', 'MajorTopicYN': 'N'})], 'DescriptorName': StringElement('Acute Kidney Injury', attributes={'UI': 'D058186', 'MajorTopicYN': 'Y'})}, {'QualifierName': [], 'DescriptorName': StringElement('Adolescent', attributes={'UI': 'D000293', 'MajorTopicYN': 'N'})}, {'QualifierName': [], 'DescriptorName': StringElement('Adult', attributes={'UI': 'D000328', 'MajorT

In [23]:
print_dict( q[0][ 'MedlineCitation' ][ 'Article' ] )

ELocationID
	 []
Language
	 ['eng']
ArticleDate
	 []
Journal
	 {'ISSN': StringElement('0147-9563', attributes={'IssnType': 'Print'}), 'JournalIssue': DictElement({'Volume': '4', 'Issue': '6', 'PubDate': {'Year': '1975', 'Season': 'Nov-Dec'}}, attributes={'CitedMedium': 'Print'}), 'Title': 'Heart & lung : the journal of critical care', 'ISOAbbreviation': 'Heart Lung'}
ArticleTitle
	 Acute renal failure.
Pagination
	 {'StartPage': '873', 'EndPage': '878', 'MedlinePgn': '873-8'}
Abstract
	 {'AbstractText': ['Acute renal failure is a life-threatening situation for which satisfactory treatment is currently available. Death is usually related to concomitant infection rather than uremia. Prompt recognition and treatment of underlying factors that predispose to renal insufficiency can frequently prevent the development of intrinsic acute renal failure.']}
AuthorList
	 ListElement([DictElement({'Identifier': [], 'AffiliationInfo': [], 'LastName': 'Freedman', 'ForeName': 'P', 'Initials': 'P'}, a

A paper's abstract can therefore be accessed with:

In [24]:
{ int(q[0]['MedlineCitation']['PMID']) : str(q[0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]) }

{349: 'Acute renal failure is a life-threatening situation for which satisfactory treatment is currently available. Death is usually related to concomitant infection rather than uremia. Prompt recognition and treatment of underlying factors that predispose to renal insufficiency can frequently prevent the development of intrinsic acute renal failure.'}

Some of the ids in our dataset refer to books from the [NCBI Bookshelf](http://www.ncbi.nlm.nih.gov/books/), a collection of freely available, downloadable, on-line versions of selected biomedical books. For such ids, `Entrez.efetch()` returns a slightly different structure, where the keys `[u'BookDocument', u'PubmedBookData']` take the place of the `[u'MedlineCitation', u'PubmedData']` keys we saw above.

### Abstracts dataset

We can now assemble a dataset mapping paper ids to their abstracts.

In [25]:
Abstracts_file = 'data/' + search_term + '_Abstracts.pkl.bz2'

In [26]:
import http.client
from collections import deque
from xml.dom import minidom
import re

def ch(node, childtype):
    return node.getElementsByTagName(childtype)[0]

if os.path.exists( Abstracts_file ):
    Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )
else:
    # `Abstracts` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Abstracts = deque()
    retrieve_per_query = 500
    
    print('Fetching Abstracts of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        # issue requests to the server, until we get the full amount of data we expect
        while True:
            try:
                #s = Entrez.read( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) )['PubmedArticle']
                s = minidom.parse( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) ).getElementsByTagName("PubmedArticle")
            except http.client.IncompleteRead:
                print('r', end='')
                continue
            break
        
        i = 0
        for p in s:
            abstr = ''
            if (p.getElementsByTagName('MedlineCitation')):
                citNode = ch(p,'MedlineCitation')
                pmid = ch(citNode,'PMID').firstChild.data
                articleNode = ch(citNode,'Article')
                if (articleNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            elif (p.getElementsByTagName('BookDocument')):
                bookNode = ch(p,'BookDocument')
                pmid = ch(bookNode,'PMID').firstChild.data
                if (bookNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            else:
                raise Exception('Unrecognized record type, for id %d (keys: %s)' % (Ids[start+i], str(p.keys())) )
            
            Abstracts.append( (int(pmid), str(abstr)) )
            i += 1
    
    # Save Abstracts, as a dictionary indexed by Ids
    Abstracts = dict( Abstracts )
    
    pickle.dump( Abstracts, bz2.BZ2File( Abstracts_file, 'wb' ) )

Taking a look at one paper's abstract:

In [47]:
print(Abstracts[500204])
print(len(Abstracts))
print(Ids[349])
dict_items_list = list(Abstracts.items())
#print('------',dict_items_list[9990:101002])

To explore the relationship between neuraminidase immunity and the degree of attenuatíon of live influenza A virus vaccines, a comparative evaluation of three Victoria/75-ts-1[E] (Vic/75-ts-1[E]) recombinant viruses in serum hemagglutination-inhibiting-negative (titer, </=1:8) adult volunteers was performed. These three ts-1[E] viruses had a similar restriction of replication at 38 degrees C in vitro, and each possessed the two attenuating genes of the ts-1[E] donor strain (13). However, Vic/75-ts-1[E] recombinants 81 and 113 possessed both Vic/75 hemagglutinin (H3(75)) and Vic/75 neuraminidase (N2(75)), whereas Vic/75-ts-1[E] recombinant 67 had Vic/75 hemagglutinin but the N2(65) neuraminidase. Vic/75-ts-1[E] recombinant 67 was significantly more attenuated than Vic/75-ts-1[E] recombinants 81 and 113 in that fewer local and systemic signs and symptoms of illness were observed in those volunteers who received clone 67. These findings were consistent with our previous observations which

## ELink: Searching for related items in NCBI Entrez

To understand how to obtain paper citations with Entrez, we will first assemble a small set of PubMed IDs, and then query for their citations.
To that end, we search here for papers published in the Nature journal with our given keyword.

In [28]:
CA_search_term = search_term+'[TIAB] AND Nature[JOUR]'
CA_ids = Entrez.read( Entrez.esearch( db="pubmed", term=CA_search_term ) )['IdList']
CA_ids

['41094140', '41050434', '41034577', '40696127', '40696125', '40681884', '40634609', '40595421', '40461670', '40437087', '40437082', '40335714', '40301615', '40240830', '40134451', '40132641', '40033147', '39984686', '39966582', '39880955']

In [29]:
CA_summ = {
    p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( CA_ids )) )
    }
CA_summ

{'41094140': ('A vaccine central in A(H5) influenza antigenic space confers broad immunity.',
  ['Kok A', 'Wilks SH', 'Tureli S', 'James SL', 'Bestebroer TM', 'Burke DF', 'Funk M', 'van der Vliet S', 'Spronken MI', 'Rijnink WF', 'Pattinson DJ', 'de Meulder D', 'Rosu ME', 'Lexmond P', 'van den Brand JMA', 'Herfst S', 'Smith DJ', 'Fouchier RAM', 'Richard M'],
  '2025',
  'Nature',
  '10.1038/s41586-025-09626-3'),
 '41050434': ('Effects of exposure to pandemic-related stressors on anxiety and mood difficulty during versus before the COVID-19 pandemic in United States Army soldiers and veterans.',
  ['Kessler RC', 'Millikan-Bell AM', 'Edwards ER', 'Gildea SM', 'King AJ', 'Liu H', 'Petukhova MV', 'Sampson NA', 'Ziobrowski HN', 'Wagner JR', 'Stein MB', 'Ursano RJ'],
  '2025',
  'Nature. Mental health',
  '10.1038/s44220-025-00505-4'),
 '41034577': ('Monoclonal antibodies protect against pandrug-resistant Klebsiella pneumoniae.',
  ['Roscioli E', 'Zucconi Galli Fonseca V', 'Bosch SS', 'Paciel

Because we restricted our search to papers in an open-access journal, you can then follow their DOIs to freely access their PDFs at the journal's website.

We will now issue calls to `Entrez.elink()` using these PubMed IDs, to retrieve the IDs of papers that cite them.
The database from which the IDs will be retrieved is [PubMed Central](http://www.ncbi.nlm.nih.gov/pmc/), a free digital database of full-text scientific literature in the biomedical and life sciences.

A complete list of the kinds of links you can retrieve with `Entrez.elink()` can be found [here](http://eutils.ncbi.nlm.nih.gov/entrez/query/static/entrezlinks.html).

In [48]:
pmid = "40301615"

results = Entrez.read(

    Entrez.elink(dbfrom="pubmed", db="pmc", LinkName="pubmed_pmc_refs", id=pmid)

)

pmc_ids = [link["Id"] for link in results[0]["LinkSetDb"][0]["Link"]]

pmc_ids

['12304356']

In [49]:
Entrez.email = "r.m.siebes@vu.nl"
CA_citing = {
    id : Entrez.read( Entrez.elink(
            cmd = "neighbor",               # ELink command mode: "neighbor", returns
                                            #     a set of UIDs in `db` linked to the input UIDs in `dbfrom`.
            dbfrom = "pubmed",              # Database containing the input UIDs: PubMed
            db = "pmc",                     # Database from which to retrieve UIDs: PubMed Central
            LinkName = "pubmed_pmc_refs",   # Name of the Entrez link to retrieve: "pubmed_pmc_refs", gets
                                            #     "Full-text articles in the PubMed Central Database that cite the current articles"
            from_uid = id                   # input UIDs
            ) )
    for id in CA_ids
    }




In [50]:
CA_citing['40301615']

[{'LinkSetDb': [{'Link': [{'Id': '12304356'}], 'DbTo': 'pmc', 'LinkName': 'pubmed_pmc_refs'}], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'pubmed', 'IdList': ['40301615']}]

We have in `CA_citing[paper_id][0]['LinkSetDb'][0]['Link']` the list of papers citing `paper_id`. To get it as just a list of ids, we can do

In [51]:
cits = [ l['Id'] for l in CA_citing['40301615'][0]['LinkSetDb'][0]['Link'] ]
cits

['12304356']

However, one more step is needed, as what we have now are PubMed Central IDs, and not PubMed IDs. Their conversion can be achieved through an additional call to `Entrez.elink()`:

In [52]:
cits_pm = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=",".join(cits)) )
cits_pm

[{'LinkSetDb': [{'Link': [{'Id': '40741302'}], 'DbTo': 'pubmed', 'LinkName': 'pmc_pubmed'}], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'pmc', 'IdList': ['12304356']}]

In [53]:
ids_map = { pmc_id : link['Id'] for (pmc_id,link) in zip(cits_pm[0]['IdList'], cits_pm[0]['LinkSetDb'][0]['Link']) }
ids_map

{'12304356': '40741302'}

And to check these papers:

In [54]:
{   p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( ids_map.values() )) )
    }

{'40741302': ("The WHO Pandemic Agreement's Missing Epistemic Architectures: Infodemics and Antimicrobial Resistance as Examples.",
  ['Ho CWL', 'Caals K'],
  '2025',
  'Asian bioethics review',
  '10.1007/s41649-025-00387-9')}

### Citations dataset

We have now seen all the steps required to assemble a dataset of citations to each of the papers in our dataset.

In [55]:
Citations_file = 'data/' + search_term + '_Citations.pkl.bz2'
Citations = []

At least one server query will be issued per paper in `Ids`. Because NCBI allows for at most 3 queries per second (see [here](http://biopython.org/DIST/docs/api/Bio.Entrez-pysrc.html#_open)), this dataset will take a long time to assemble. Should you need to interrupt it for some reason, or the connection fail at some point, it is safe to just rerun the cell below until all data is collected.

In [ ]:
import http.client
from urllib.error import HTTPError

if Citations == [] and os.path.exists( Citations_file ):
    Citations = pickle.load( bz2.BZ2File( Citations_file, 'rb' ) )

if len(Citations) < len(Ids):
    
    i = len(Citations)
    checkpoint = int(len(Ids) / 1000) + 1      # save to hard drive at every 0.1% of Ids fetched
    
    for pm_id in Ids[i:]:               # either starts from index 0, or resumes from where we previously left off
        
        while True:
            try:
                # query for papers archived in PubMed Central that cite the paper with PubMed ID `pm_id`
                c = Entrez.read( Entrez.elink( dbfrom = "pubmed", db="pmc", LinkName = "pubmed_pmc_refs", id=str(pm_id) ) )
                
                c = c[0]['LinkSetDb']
                if len(c) == 0:
                    # no citations found for the current paper
                    c = []
                    #print("no links for",pm_id)
                else:
                    c = [ l['Id'] for l in c[0]['Link'] ]
                    
                    # convert citations from PubMed Central IDs to PubMed IDs
                    p = []
                    retrieve_per_query = 100
                    for start in range( 0, len(c), retrieve_per_query ):
                        query_ids = ','.join( c[start : start+retrieve_per_query] )
                        r = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=query_ids ) )
                        # select the IDs. If no matching PubMed ID was found, [] is returned instead
                        p.extend( [] if r[0]['LinkSetDb']==[] else [ int(link['Id']) for link in r[0]['LinkSetDb'][0]['Link'] ] )
                    c = p
            
            except http.client.BadStatusLine:
                # Presumably, the server closed the connection before sending a valid response. Retry until we have the data.
                print('r')
                continue
            except http.client.IncompleteRead:
                print('incompleteRead, continuing')
                continue
            except HTTPError:
                print('r')
                continue
            break
        
        Citations.append( (pm_id, c) )
       # print(pm_id,' :',c)
        if (i % 1000 == 0):
            print('')
            print(i, end='')
        if (i % 10 == 0):
            print('.', end='')
        i += 1
        
        if i % checkpoint == 0:
            print('\tsaving at checkpoint', i)
            pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )
    
    print('\n done.')
    
    # Save Citations, as a dictionary indexed by Ids
    Citations = dict( Citations )
    
    pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )

........	saving at checkpoint 20097
........	saving at checkpoint 20174
........	saving at checkpoint 20251
.......	saving at checkpoint 20328
........	saving at checkpoint 20405
........	saving at checkpoint 20482
.......	saving at checkpoint 20559
........	saving at checkpoint 20636
........	saving at checkpoint 20713


To see that we have indeed obtained the data we expected, you can match the ids below, with the ids listed at the end of last section.

In [ ]:
print(len(Citations))
Citations[84842]
Summaries[10576728]


## Where do we go from here?

Running the code above generates multiple local files, containing the datasets we'll be working with. Loading them into memory is a matter of just issuing a call like<br>
``data = pickle.load( bz2.BZ2File( data_file, 'rb' ) )``.

The Entrez module will therefore no longer be needed, unless you wish to extend your data processing with additional information retrieved from PubMed.

Should you be interested in looking at alternative ways to handle the data, have a look at the [sqlite3](http://docs.python.org/3/library/sqlite3.html) module included in Python's standard library, or [Pandas](http://pandas.pydata.org/), the Python Data Analysis Library.